In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <center>Team 2 Unsupervised Machine Learning 

Team 4 Unsupervised learning team members: 
Charles Maponya, 
Chuene Mokgokong,
Kgomotso Moepe,
Lesedi Madumo, 
Thabisile Obi, 
Tumelo Malebo

[Trello board ](https://trello.com/b/dVQJqi5C/team4unsupervised)

## Problem Statement 
Accurately predict unseen movie ratings gathered from thousands of users based on their historic preferences. 

The objective of this notebook is to construct a recommendation algorithm based on content and collaborative filtering, capable of accurately predicting how a user will rate a movie they have not watched yet based on their historical preference.   

![image.png](https://manofmany.com/wp-content/uploads/2020/04/Veboli-new-2.jpg)

# Start Comet experiment

In [ ]:
!pip install comet_ml

In [ ]:
# import comet_ml at the top of your file
from comet_ml import Experiment

In [ ]:
# Create an experiment with your api key:
experiment = Experiment(
    api_key=,
    project_name=,
    workspace=,
)

# Table of contents:

### [1. Introduction](#introduction)

### [2. Import libaries and datasets](#libaries)

### [3. Exploratory Data Analysis](#eda)

* [Summary statistics](#sub1)
* [Visualizing the dataframes](#sub2)
* [Visualizing the null values for each dataframe](#sub3)
* [Visualizing common users](#sub4)
* [Exploring movie genres](#sub5) 
* [Exploring the movies dataframe](#sub6)
* [Word cloud](#sub7)
* [Publishing years](#sub8)
* [Budget](#sub9)

### [4. Prepocessing](#prep)


### [5. Modelling](#mod)

* [Content-Based Filtering Recommendation](#CB)
* [Collaborative-Based Filtering Reccomendation](#CB1)

### [6. Evaluation](#eva)

### [7. Conclusion](#conc)


# [1. Introduction](#introduction)

Recommender systems are systems that are designed to recommend things to the user based on many different factors. These systems predict the most likely product that the user is most likely to purchase and are of interest. Companies like Netflix and Amazon use recommender systems to help their users to identify the correct product or movies for them.

Recommender systems are an important class of machine learning algorithms that offer relevant suggestions to users. The suggested items are as relevant to the user as possible so that the user can engage with those items: YouTube videos, news articles, online products, movie and series recommendation.

Items are ranked according to their relevancy, and the most relevant ones are shown to the user. The relevance is determined by the recommender system, mainly based on historical data. For example, If you've recently watched YouTube videos about elephants, then YouTube is going to start showing you many elephant videos with similar titles and themes. 
Recommender systems are generally divided into two main categories: collaborative filtering and content-based systems.

![image.png](https://miro.medium.com/max/690/1*G4h4fOX6bCJhdmbsXDL0PA.png)
 
Both users and service providers have benefited from these kinds of systems. Intelligent algorithms can help viewers find great titles from tens of thousands of options. This notebook will construct a recommendation algorithm based on content and collaborative filtering, capable of accurately predicting how a user will rate a movie they have not yet viewed based on their historic preferences.

Providing an accurate and robust solution will have immense economic potential, with users of the system being exposed to content they would like to view or purchase - generating revenue and platform affinity.

# [2. Import libaries and datasets](#libaries)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from wordcloud import WordCloud, STOPWORDS
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
        
import surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
import time
from surprise import SVD
from surprise import accuracy
import re
import plotly.express as px
import scipy as sp
from wordcloud import WordCloud, STOPWORDS
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline  import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, Normalizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

### Loading all Necessary Data sets

In [ ]:
data_folder = '/kaggle/input/edsa-recommender-system-predict/'
#data_folder = 'C:/Users/ADMIN/Desktop/unsupervised_project/'
train_df = pd.read_csv(data_folder + 'train.csv')
test_df = pd.read_csv(data_folder + 'test.csv')
tags_df = pd.read_csv(data_folder + 'tags.csv')
movies_df = pd.read_csv(data_folder + 'movies.csv')
links_df = pd.read_csv(data_folder + 'links.csv')
imdb_df = pd.read_csv(data_folder + 'imdb_data.csv')
genome_tags = pd.read_csv(data_folder + 'genome_tags.csv')
genome_score = pd.read_csv(data_folder + 'genome_scores.csv')

# [3. Exploratory Data Analysis](#eda)

Exploratory data analysis is an approach to analyzing data sets to summarize their main characteristics, often with visual methods. EDA is the critical process of performing initial investigations on data to discover patterns, to spot anomalies, to test hypothesis and to check assumptions with the help of summary statistics and graphical representations.

A statistical model can be used or not, but primarily EDA is for seeing what the data can tell us beyond the formal modelling or hypothesis testing task.

It is a good practice to understand the data first and try to gather as many insights from it. EDA is all about making sense of data in hand, before getting them dirty with it, which will be done below.

### [Summary statistics](#sub1)
Pandas dataframe.info() function is used to get a concise summary of the dataframe. It comes really handy when doing exploratory analysis of the data

In [ ]:
## Creating a summary stats function
def Summary(df):
    return df.info()

In [ ]:
Summary(train_df)

In [ ]:
Summary(test_df)

In [ ]:
Summary(tags_df)

In [ ]:
Summary(movies_df)

In [ ]:
Summary(links_df)

In [ ]:
Summary(imdb_df)

In [ ]:
Summary(genome_tags)

In [ ]:
Summary(genome_score)

Most of the DataFrames have a Dtype of int64 and float64, which is an indication for numeric values. However, some DataFrames also have Dtype of an object, which is an indication of non-numeric character. The DataFrames which have a Dtype include tags_df; movies_df; imdb_df; and genome_df. 

##### Getting The shapes of our data sets

In [ ]:
print('train data:',train_df.shape) 
print('test data:',test_df.shape)
print('tags data:',tags_df.shape)
print("Movies data:",movies_df.shape)
print('links data:',links_df.shape)
print('imdb data:',imdb_df.shape)
print('genome tags data:',genome_tags.shape)
print('genome scores data:',genome_score.shape)

### [Visualizing the dataframes](#sub2)

In [ ]:
# Created a Data Frame outlining the size of our data
dataframes = ['train_df', 'test_df', 'tags_df', 'imdb_df',
              'links_df', 'movies_df', 'genome_tags', 'genome_score']
sizes = [len(train_df), len(test_df), len(tags_df),
         len(imdb_df), len(links_df), len(movies_df),
         len(genome_tags), len(genome_score)]
total_size_df = pd.DataFrame(list(zip(dataframes, sizes)),
                             columns=['dataframe', 'sizes'])
total_size_df

In [ ]:
total_size_df = total_size_df[total_size_df['sizes'] > 100000]
total_size_df

In [ ]:
new_row = {'dataframe': 'other', 'sizes': 180530}
total_size_df = total_size_df.append(new_row,
                                     ignore_index=True)
total_size_df

In [ ]:
explodeTuple = (0.05, 0.04, 0.05, 0.04, 0.6)
fig1, ax1 = plt.subplots(figsize=(14,7))
ax1.pie(total_size_df['sizes'].values,
        labels=total_size_df['dataframe'].values,
        startangle=90, autopct='%1.1f%%',
        explode=explodeTuple)
ax1.axis('equal')
plt.title('Distribution of overall Data Frames')
plt.show()

In [ ]:
len_list = [['train_df', len(train_df)], ['tags_df', len(tags_df)],
            ['imdb_df', len(imdb_df)], ['links_df', len(links_df)],
            ['movies_df', len(movies_df)],
            ['genome_tags', len(genome_tags)],
            ['genome_score', len(genome_score)]]
len_df = pd.DataFrame(len_list,
                      columns=['Dataset', 'Size'])
fig = px.bar(len_df, x=len_df['Dataset'],
             y=len_df['Size'],
             color=len_df['Dataset'],
             title='Distribution of overall Data Frames')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()


The DataFrame's that are visualised in the pie chart are the DataFrames with a size larger than 100000. The DataFrames that had a larger size than 100000 include the train_df; test_df; tags_df; genome_score and other. While the DataFrame's that had a size smaller than 100000 were combined into a DataFrame 'other', these DataFrames are imdb_df; links_df, movies_df; and genome_tags. 

In the pie chart, it is visually seen that the genome_score DataFrame size (48.9%) accounts for almost 50% of the DataFrame sizes. Followed by train_df account for 31.4%. The DataFrame with the least size is the other, with an attribute of only 0.6%.

In the bar graph, the precise unequal distribution of DataFrame sizes is evident. The genome_score DataFrame has the largest size, followed by the train_df. The difference in distribution sizes is clear, where the other DataFrame's bars aren't visually evident because of the large difference between the dimensions.

### [Visualizing the null values for each dataframe](#sub3)

In [ ]:
# Obtaining the total null values in each Data Frames columns
train_count = pd.DataFrame(train_df.isnull().sum())
test_count = pd.DataFrame(test_df.isnull().sum())
tags_count = pd.DataFrame(tags_df.isnull().sum())
movies_count = pd.DataFrame(movies_df.isnull().sum())
links_count = pd.DataFrame(links_df.isnull().sum())
imdb_count = pd.DataFrame(imdb_df.isnull().sum())
genomet_count = pd.DataFrame(genome_tags.isnull().sum())
genomes_count = pd.DataFrame(genome_score.isnull().sum())

In [ ]:
train_count

In [ ]:
test_count

In [ ]:
plt.bar(tags_count.index,
        tags_count.values.reshape(len(tags_count), ),
        color='red')
plt.xlabel('column_name')
plt.ylabel('count')
plt.title('Null value count in tags_df')
plt.show()

In [ ]:
movies_count

In [ ]:
links_count

In [ ]:
plt.bar(imdb_count.index,
        imdb_count.values.reshape(len(imdb_count), ),
        color='orange')
plt.xlabel('column_name')
plt.ylabel('count')
plt.title('Null value count in imdb_df')
plt.show()

It is essential to check whether your dataset has missing values. The real-world data often has a lot of missing values. The cause of missing values can be data corruption or failure to record data. The handling of missing data is critical during the preprocessing of the dataset, as many machine learning algorithms do not support missing values.

The DataFrame's with null values include the tags_df and imdb_df.
In the tags_df, the column with null values is the tag column as seen by the red bar in the Null value count in tags_df bar graph. 
In the imdb_df, there are five columns with null values, with the budget column having the most null values of over 17500. 
There are several methods for dealing with missing values. This includes:

1. Deleting Rows with missing values
2. Impute missing values for continuous variable
3. Impute missing values for categorical variable
4. Other Imputation Methods
5. Using Algorithms that support missing values
6. Prediction of missing values
7. Imputation using Deep Learning Library — Datawig

### [Visualizing common users](#sub4)

In [ ]:
# To find the number of times a user rated a movie, we create a data frame with the count by userId
train_user = pd.DataFrame(
    train_df['userId'].value_counts()).reset_index()
train_user.rename(columns={'index':'userId','userId':'count'},
                  inplace=True)
train_user.head()

In [ ]:
# Grouping the users within a certain range aided us in determining the common userId's and the new ones.
group_one = train_user.loc[(train_user['count'] > 0) & 
            (train_user['count'] < 50),
            'userId'].value_counts().sum()
group_two = train_user.loc[(train_user['count'] >= 50) & 
            (train_user['count'] < 500),
            'userId'].value_counts().sum()
group_three = train_user.loc[(train_user['count'] >= 500) & 
            (train_user['count'] < 1000),
            'userId'].value_counts().sum()
group_four = train_user.loc[(train_user['count'] >= 1000) & 
            (train_user['count'] < 1500),
            'userId'].value_counts().sum()
group_five = train_user.loc[(train_user['count'] >= 1500),
            'userId'].value_counts().sum()

In [ ]:
# To give us insight in the spread, we used figures to determine the spread.
trial_error = np.array([['group_one', group_one,
                         'between 1 and 50'],
                        ['group_two', group_two,
                         'between 50 and 500'],
                        ['group_three', group_three,
                         'between 500 and 1000'],
                        ['group_four', group_four,
                         'between 1000 and 1500'],
                        ['group_five', group_five,
                         'greater than 1500']])
trial_error_df = pd.DataFrame({'group': trial_error[:, 0],
                               'userId_grouping': trial_error[:, 1],
                               'explanation': trial_error[:, 2]})
fig = px.bar(trial_error_df,
             x=trial_error_df["group"],
             y=trial_error_df["userId_grouping"],
             color=trial_error_df["group"],
             title='Grouped Rating Distribustion')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()
trial_error_df


The user Id's are grouped by the rating counts in a grouping range illustrated in the DataFrame above.  In the Grouped Rating Distribution bar graph, it is visually displayed that there is unequal distribution.  The distribution is skewed to the left, with the majority of the user ids in the rating count range between 1 and 50. At the same time, the last group has only a value count of 61, which is a significant difference from group one with a value count of 110 010.

In [ ]:
def user_ratings_count(df, n):
    plt.figure(figsize=(14,7))
    data = df['userId'].value_counts().head(n)
    ax = sns.barplot(x = data.index, y = data, order= data.index, palette='CMRmap', edgecolor="black")
    for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), fontsize=11, ha='center', va='bottom')
    plt.title(f'Top {n} Users by Number of Ratings', fontsize=14)
    plt.xlabel('User ID')
    plt.ylabel('Number of Ratings')
    plt.show()

In [ ]:
user_ratings_count(train_df,10)

<b color='darkslateblue'>Filtering out user 72315 because his/her number of raings is too extreme and he/she is an oulier</b>

In [ ]:
user_ratings_count(train_df[train_df['userId'] !=72315],10)

There are certain things which, if they are not done in the EDA phase, can affect further statistical / Machine Learning modelling. One of the things is to find outliers. Once user 72315 was removed from the top 10 users by the number of ratings, a more uniform pattern of voting is evident in the bar graph. 

In statistics, an outlier is an observation point that is distant from other observations. The definition suggests to us that an outlier is something which is an odd-one-out or the one that is different from the crowd. Some statisticians define outliers as 'having a different underlying behaviour than the rest of the data'.  Alternatively, an outlier is a data point that is distant from other points.
The Top 10 User by Number of Ratings graph that includes user 72315 is not a representation of an imbalanced dataset. An imbalanced data set in terms of machine learning is where one class label has far fewer samples compared to another class label. In this case, user 72315 deviates significantly from the rest of the users. 

![image.png](https://datascience.foundation/img/pdf_images/knowing_all_about_outliers_in_machine_learning_sample_points_in_green_are_near_to_each_other.jpg)

From the image above we can see that the sample points in Green are close to each other, whereas the two sample points in Red are far apart from them. These red sample points are outliers.
Machine learning algorithms are sensitive to the range and distribution of attribute values. Data outliers can spoil and mislead the training process resulting in longer  training times, less accurate models and ultimately more inferior results.

### [Exploring Movie Genres](#sub5)

In [ ]:
genres = pd.DataFrame(movies_df['genres'].
                      str.split("|").
                      tolist(),
                      index=movies_df['movieId']).stack()
genres = genres.reset_index([0, 'movieId'])
genres.columns = ['movieId', 'Genre']
genres.head()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
sns.countplot(x='Genre',
              data=genres,
              palette='CMRmap',
              order=genres['Genre'].
              value_counts().index)
plt.xticks(rotation=90)
plt.xlabel('Genre', size=20)
plt.ylabel('Count', size=20)
plt.title('Distribution of Movie Genres', size=25)
plt.show()

The top 3 most popular movie genres include drama, comedy and thriller. 
A Genre consists of four elements or parts: character, story, plot and setting. And often people will state that a particular movie had a good plot or an intriguing story. What people are referring to is that they enjoyed the characters, the problems/conflict the characters got into, and how the characters got out of the problems and conflict. The drama genre could be the most popular because it caters the character development in the plot, often overcoming dome form of challenge and conflicts, i.e. human struggles.  According to Hayley Mckenzie, drama delivers the emotional and relational development of realistic characters in a realistic setting. It offers intense character development and tells an honest story of human struggle. And this could be the possible reason why drama is a popular genre. 

### [Exploring the Movies data](#sub6)

In [ ]:
movies=pd.merge(train_df, movies_df,on='movieId',how='inner')
movies.head()

In [ ]:
full_movies = pd.merge(movies,imdb_df,on='movieId',how='inner')
full_movies.head()

In [ ]:
def top_n_plot_by_ratings(df,column, n):
    plt.figure(figsize=(14,7))
    data = df[str(column)].value_counts().head(n)
    ax = sns.barplot(x = data.index, y = data, order= data.index, palette='CMRmap', edgecolor="black")
    for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), fontsize=11, ha='center', va='bottom')
    plt.title(f'Top {n} {column.title()} by Number of Ratings', fontsize=14)
    plt.xlabel(column.title())
    plt.ylabel('Number of Ratings')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
top_n_plot_by_ratings(movies,'title',15)

In the Top 15 Title by Number of Ratings bar graph, all the movies are prior the year 2001, with 14 of them released in the 19th century. 

The top three are Shawshank Redemption 1994, Forest Grump 1994 and Pulp Fiction 1994.  All three movies fall under the popular drama genre and are American.
The Shawshank Redemption is a 1994 American drama film written and directed by Frank Darabont, based on the 1982 Stephen King novella Rita Hayworth and Shawshank Redemption. 
Pulp Fiction is a 1994 American neo-noir black comedy crime film written and directed by Quentin Tarantino, who conceived it with Roger Avary.
Forrest Gump is a 1994 American romantic comedy-drama film directed by Robert Zemeckis and written by Eric Roth.

In [ ]:
# Wordcloud of movie titles
movies_word = movies_df['title'] = movies_df['title'].astype('str')
movies_wordcloud = ' '.join(movies_word)
title_wordcloud = WordCloud(stopwords = STOPWORDS,
                            background_color = 'White',
                            height = 1200,
                            width = 900).generate(movies_wordcloud)
plt.figure(figsize = (14,7), facecolor=None)
plt.imshow(title_wordcloud)
plt.axis('off')
plt.title('Distribution of words from movie titles')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
top_n_plot_by_ratings(movies,'rating',10)

The most common rating score that is give is 4.0, followed by 3.0. The least common score that is given by usrs is 0.5.

In [ ]:
top_n_plot_by_ratings(full_movies,'director',15)

Quentin Tarantino is the top director of the number of ratings. Quentin Tarantino is one of the world's most renowned writer/directors. Each of his films is highly anticipated, as he draws upon the subjects and themes of previous films to produce a fresh storyline. Three of the most significant works from Quentin Tarantino are Reservoir Dogs, True Romance and Pulp Fiction. 


![image.png](https://img.theculturetrip.com/1440x/smart/images/56-3992968-1001514452-58f5b96a08.jpg)

And as seen in the by the Top 15 Title by Number of Ratings graph, Pulp Fiction is one of the top 3 most voted films. 
For best director he was nominated for: 

1994	Pulp Fiction	Nominated

2009	Inglourious Basterds	Nominated

2019	Once Upon a Time in Hollywood	Nominated

Other competing directors include Michael Crichton and J.R.R. Tolkien. However, Quentin Tarantino does have a more considerable leading as compared to other directors in the graph. 

In [ ]:
movieRatingDistGroup = train_df['rating'].value_counts().sort_index().reset_index()
fig, ax = plt.subplots(figsize=(14,7))
sns.barplot(data=movieRatingDistGroup, x='index', y='rating', palette="CMRmap", edgecolor="black", ax=ax)
ax.set_xlabel("Rating")
ax.set_ylabel('Number of Users')
ax.set_yticklabels(['{:,}'.format(int(x)) for x in ax.get_yticks().tolist()])
total = float(movieRatingDistGroup['rating'].sum())
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2., height+350, '{0:.2%}'.format(height/total), fontsize=11, ha="center", va='bottom')
plt.title('Number of Users Per Rating', fontsize=14)
plt.show()

Most of the users are weighted within the score range of 3.0 - 5.0, with the most users being weighted in the 4.0 score, accounting for 26.53% of the users. This indicates that most of the users give ratings above 2.5, and with fewer users giving a score of 0.5 which only only consists of 1.58% of the users. Similiarly with the score of 1.5 only accounting to 1.60% of users.

In [ ]:
def count_directors(df, count = 10):
    """
    Function to count the most common dircetors in a DataFrame:
    Parameters
    ----------
        df (DataFrame): input dataframe containing imdb metadata
        count (int): filter directors with fewer than count films
        
    Returns
    -------
        directors (DataFrame): output DataFrame
    Examples
    --------
        >>> df = pd.DataFrame({'imdbid':[0,1,2,3,4,5], 'director': [A,B,A,C,B]})
        >>> count_directors(df, count = 1)
            |index|director|count|
            |0|A|2|
            |1|B|2|
            |2|C|1|
    """
    directors = pd.DataFrame(df['director'].value_counts()).reset_index()
    directors.columns = ['director', 'count']
    # Lets only take directors who have made 10 or more movies otherwise we will have to analyze 11000 directors
    directors = directors[directors['count']>=count]
    return directors.sort_values('count', ascending = False)

In [ ]:
def feature_count(df, column):
    plt.figure(figsize=(14,7))
    ax = sns.barplot(x = df[f'{column}'], y= df['count'], palette='brg')
    for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), fontsize=11, ha='center', va='bottom')
    plt.title(f'Number of Movies Per {column}', fontsize=14)
    plt.xlabel(f'{column}')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
directors = count_directors(imdb_df)
feature_count(directors.head(15), 'director')

In [ ]:
feature_count(directors[directors['director']!='See full summary'].head(15), 'director')

In the Number of Movies Per Director bar graph, the leading director who has produced the most movies is 'See full summary' with a count of 28. However, 'see the full summary' is not a director! 
Luc Besson and Woody Allen are tied with a value count of producing 26 movies and followed by Stephan King with 24.  They are the only producers in the dataset with over 20 movie productions. 

### [Word Clouds](#sub7)

In [ ]:
comment_words = ''
stopwords = set(STOPWORDS)

# iterate through the csv file
for val in tags_df['tag']:

    # typecaste each val to string
    val = str(val)

    # split the value
    tokens = val.split()

    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width=1200, height=900,
                      colormap='winter',
                      background_color='white',
                      stopwords=stopwords,collocations=False,
                      min_font_size=10).generate(comment_words)

# plot the WordCloud image
plt.figure(figsize=(14, 7), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.title('Distribution of words in the tags data frame by Tags')
plt.tight_layout(pad=0)

plt.show()


In [ ]:
value_count = pd.DataFrame(tags_df['tag'].
                           value_counts()).reset_index()
value_count.rename(columns = {'index': 'genre', 'tag': 'count'},
                   inplace = True)

In [ ]:
value_count.head()

In [ ]:
genre_count = value_count.head(20)
plt.figure(figsize=(14,7))
ax = sns.barplot(x = genre_count['genre'], y= genre_count['count'], palette='CMRmap')
for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), fontsize=11, ha='center', va='bottom')
plt.title('Number of times a genre tag appears', fontsize=14)
plt.xlabel('Genre tag')
plt.ylabel('Genre tag Count')
plt.xticks(rotation=90)
plt.show()

The most popular words in the world cloud include book, comedy, ending, based, dark and sci-fi.  
The three most popular genres that appear in tags_df('tags') are sci-fi, atmospheric, and action. The sci-fi genre is a speculative fiction that typically deals with imaginative and futuristic concepts such as advanced science and technology, space exploration, time travel, parallel universes, and extraterrestrial life.  While atmospheric films tap in the sentimental human memory may be portals to another time and space.  For instance, scenes that give an opportunity of temporal teleportation for the nostalgic visionary. The action genre has the protagonist or protagonists who thrust into a series of events that typically include violence, extended fighting, physical feats, rescues and frantic chases.

### [Publishing Years](#sub8)

In [ ]:
dates = []
for title in movies_df['title']:
    if title[-1] == " ":
        year = title[-6: -2]
        try:
            dates.append(int(year))
        except:
            dates.append(9999)
    else:
        year = title[-5: -1]
        try:
            dates.append(int(year))
        except:
            dates.append(9999)

movies_df['Publish Year'] = dates


In [ ]:
dates = []
for title in movies_df['title']:
    if title[-1] == " ":
        year = title[-6: -2]
        try:
            dates.append(int(year))
        except:
            dates.append(9999)
    else:
        year = title[-5: -1]
        try:
            dates.append(int(year))
        except:
            dates.append(9999)

movies_df['Publish Year'] = dates



In [ ]:
len(movies_df)

In [ ]:
len(movies_df[movies_df['Publish Year'] == 9999])

In [ ]:
movies_df[(movies_df['Publish Year'] > 1888) &
          (movies_df['Publish Year'] < 2021)]

In [ ]:
dataset = pd.DataFrame(movies_df['Publish Year'].
                       value_counts()).reset_index()
dataset.rename(columns={'index': 'year', 'Publish Year': 'count'},
               inplace=True)
dataset.head()

In [ ]:
year_dataset = dataset[(dataset['year']>1888) & (dataset['year']<2021)].sort_values(by='count',ascending=False).head(50)
plt.figure(figsize=(14,7))
ax = sns.barplot(x = year_dataset['year'], y= year_dataset['count'], order=year_dataset['year'], palette='CMRmap')
#for p in ax.patches:
#       ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), fontsize=11, ha='center', va='bottom')
plt.title('Number of Movies Released Per year', fontsize=14)
plt.xlabel('year')
plt.ylabel('Released Movie Count')
plt.xticks(rotation=90)
plt.show()

In the Number of  Movies Released Per year graph, we are able to visually see an major increase in movie releases in the 21st century. 

### [Budget](#sub9)

In [ ]:
new_l = list(imdb_df['budget'])
print(type(new_l[9]))

imdb_df['runtime'] = imdb_df['runtime'].fillna(imdb_df['runtime'].mean())
imdb_df.isnull().sum() #data cleaning
imdb_df.head()
imdb_df['budget'] = imdb_df['budget'].str.replace('[\,]', '', regex=True)

In [ ]:
def clean_txt(text):
    text = re.sub(r'[0-9]+', "", str(text))
    return text
imdb_df['currency'] = imdb_df['budget'].apply(clean_txt)
imdb_df.head()

In [ ]:
currencies = list(imdb_df['currency'])
# Number of currencies
len(set(currencies))-1

currencies_count_df = pd.DataFrame(imdb_df['currency'].
                                   value_counts()).reset_index()
currencies_count_df.rename(columns={'index': 'currency', 'currency': 'count'},
                           inplace=True)
currencies_count_df.head()

fig = px.bar(currencies_count_df, x=currencies_count_df['currency'],
             y=currencies_count_df['count'],
             color=currencies_count_df['currency'],
             title='Currency Type Distribution')
fig.show()

# [4. Prepocessing](#prep)

## Multi-Dimensional Scaling 

Multidimensional scaling (MDS) is a technique for visualizing distances between objects on a map, where the distance is known between pairs of the objects.

In [ ]:
def data_scaler(df):
    """
    Scales data.
    """
    scaler = StandardScaler(with_std=True)
    scaled_data = scaler.fit_transform(df)
    return scaled_data

In [ ]:
genome= genome_score[:10000000]

In [ ]:
scaled_genome = data_scaler(genome.sample(frac=0.0001))

In [ ]:
tsne = TSNE(3, n_jobs = -1, verbose = 2, perplexity = 10, learning_rate = 0.1)
tsne.fit(scaled_genome)

In [ ]:
Axes3D

fig = plt.figure(figsize=(14, 7))

# Add 3D scatter plot
ax = fig.add_subplot(projection='3d')
ax.scatter(tsne.embedding_[:,0], tsne.embedding_[:,1], tsne.embedding_[:,2])
plt.show()

In [ ]:
fig = plt.figure(figsize=(14, 7))
sns.scatterplot(x = tsne.embedding_[:,0], y = tsne.embedding_[:,1], size=tsne.embedding_[:,2])
plt.show()

In [ ]:
# Below is the dataframe we will be altering.
working_train = train_df.drop(columns='timestamp')

In [ ]:
df_work = working_train.set_index('movieId').join([movies_df[['movieId',
                                                           'genres']]
                                                   .set_index('movieId'),
                                                   imdb_df[['movieId',
                                                         'title_cast',
                                                         'director',
                                                         'plot_keywords']].
                                                   set_index('movieId')],
                                                  how='left').reset_index()
df_work.head()


In [ ]:
def preprocessor_train(df):
    working_train = df.copy()

    # Merge
    df_work = working_train.set_index('movieId').join([movies_df
                                                       [['movieId', 'genres']].
                                                       set_index('movieId'),
                                                       imdb_df[['movieId',
                                                             'title_cast',
                                                             'director',
                                                             'plot_keywords']].
                                                       set_index('movieId')],
                                                      how='left').reset_index()

# '(no genre listed)' is an equivalent of a missing value in the column genres

    df_work['genres'] = ['' if x == '(no genres listed)' else x for x in df_work['genres']]

    # filling missing values with 'nothing'... (emptying...?)
    df_work.fillna('', inplace=True)

    for col in df_work.select_dtypes('object').columns: # selecting 'object' columns

        # removing white space
        df_work[col] = [''.join(x.split()) for x in df_work[col]]

        # substituting '|' with a white space
        df_work[col] = [' '.join(x.split('|')) for x in df_work[col]]

    # joining the features of interest
    df_work['corpus'] =  df_work[df_work.select_dtypes('object').columns].apply(lambda x: ' '.join(x), axis=1)
    return df_work[['movieId', 'userId', 'corpus', 'rating']]


In [ ]:
def preprocessor_test(df):
    working_train = df.copy()

    # Merge
    df_work = working_train.set_index('movieId').join([movies_df
                                                       [['movieId', 'genres']].
                                                       set_index('movieId'),
                                                       imdb_df[['movieId',
                                                             'title_cast',
                                                             'director',
                                                             'plot_keywords']].
                                                       set_index('movieId')],
                                                      how='left').reset_index()

# '(no genre listed)' is an equivalent of a missing value in the column genres

    df_work['genres'] = ['' if x == '(no genres listed)' else x for x in df_work['genres']]

    # filling missing values with 'nothing'... (emptying...?)
    df_work.fillna('', inplace=True)

    for col in df_work.select_dtypes('object').columns: # selecting 'object' columns

        # removing white space
        df_work[col] = [''.join(x.split()) for x in df_work[col]]

        # substituting '|' with a white space
        df_work[col] = [' '.join(x.split('|')) for x in df_work[col]]

    # joining the features of interest
    df_work['corpus'] =  df_work[df_work.select_dtypes('object').columns].apply(lambda x: ' '.join(x), axis=1)
    return df_work[['movieId', 'userId', 'corpus']]

In [ ]:
Test = preprocessor_test(test_df)

In [ ]:
X = preprocessor_train(train_df.drop(columns=['timestamp'])) # DO NOT RUN THIS ON LOCAL COMPUTER


In [ ]:
X1 = X.drop(columns=['rating', 'userId', 'movieId'])
T1 = Test.drop(columns=['userId', 'movieId'])


In [ ]:
y = X['rating']

In [ ]:
cv = CountVectorizer()
X_mat =cv.fit_transform(X1['corpus'])

In [ ]:
Test_mat = cv.transform(T1['corpus'])

In [ ]:
X_mat.shape

# [5. Modelling](#mod)

### [Content-Based Filtering Recommendation](#CB)

Collaborative filtering addresses some of the limitations of content-based filtering; collaborative filtering uses similarities between users and items simultaneously to provide recommendations. This allows for serendipitous recommendations; that is, collaborative filtering models can recommend an item to user A based on the interests of a similar user B. Furthermore, the embeddings can be learned automatically, without relying on hand-engineering of features.

#### Linear Regression Model
Simple linear regression is a statistical method that shows the relationship between two continuous variables. This is represented by a straight line with the equation:
$$ y = a + bx$$   
where $a$ is the intercept of the line with the y-axis, and $b$ is the gradient.  
The independent variable ($x$) is also known as the predictor and the dependent variable ($y$) is known as the target.

In [ ]:
estimators2 = []
estimators2.append(('standardize',
                    StandardScaler(with_mean=False)))
estimators2.append(('mod',
                    LinearRegression()))
model = Pipeline(estimators2)
model.fit(X_mat, y)

We merged each movie with its genre, casts, director and plot keywords.

We wanted to find out how similar the test movies are to any movie the user has rated before, then rate the 'test movie' by the highest similarity in any rating category.

Therefore, for each user, we categorised the movies viewers have watched by rating.

Results for the linear regression model predicted a RMSE of 1.06. We can conclude therefore that there are not enough variables to lower the RMSE though this still proved to be the best performing model for content based filtering.


#### Cosine-similarity

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
# Break up the big genre string into a string array
genre = movies_df['genres'].str.split('|')
genre = movies_df['genres'].fillna("").astype('str')
sample_genre = genre.head(20000)   # remove sample on EC2 to acess the whole dataset
#create a Tf_idf vectorizer
tf = TfidfVectorizer(analyzer = 'word',
                     ngram_range = (1, 3),
                     min_df = 0,
                     stop_words = 'english')
vecto = tf.fit_transform(sample_genre)     # replace sample_genre with genre
# cosine similarities
cosine_sim = linear_kernel(vecto, vecto)
# The movie_recommendation function
def movie_recommendations(movie):
    # set the movie title as the new index
    movie_index=pd.Series(movies_df.index,
                          index = movies_df['title'])
    # generate  similarities between the movie title and movie index based on genre
    similarities = list(enumerate(cosine_sim[movie_index[movie]]))
    similarities = sorted(similarities, key = lambda x: x[1],
                          reverse = True)[1:11]
    movie_recommendation = movies_df['title'].iloc[[i[0] for i in similarities]]
    return movie_recommendation

In [ ]:
movie_recommendations('Into the West (2005)')

The Content-Based Recommendation system computes similarity between movies based on movie genres using the selected movie as a baseline. The TF-IDF vectorizer was used to find the relationship in terms of relative importance of the movie.
The systems make use of Cosine Similarity to compute numeric quantities that highlights the similarities between the movies (one as a baseline).
A function was assembled to estimate top 10 movies that are similar to the base movie using genre as a basis.
The function was passed an input (‘Into the West (2005)’), a baseline and generated top 10 similar movies using cosine similarity score computed above.

### [Collaborative-Based Filtering Reccomendation](#CB1)
Collaborative filtering addresses some of the limitations of content-based filtering; collaborative filtering uses similarities between users and items simultaneously to provide recommendations. This allows for serendipitous recommendations; that is, collaborative filtering models can recommend an item to user A based on the interests of a similar user B. Furthermore, the embeddings can be learned automatically, without relying on hand-engineering of features.

In [ ]:
# Load the 1M dataset
train_df.drop('timestamp', axis=1)
train_subset = train_df[:1000000]
reader = Reader(rating_scale=(train_subset['rating'].min(), train_subset['rating'].max()))
data = Dataset.load_from_df(train_subset[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=42)

#### Non-Negative Matrix Factorization Algorithm

Non-negative matrix factorization, also non-negative matrix approximation is a group of algorithms in multivariate analysis and linear algebra where a matrix V is factorized into two matrices W and H, with the property that all three matrices have no negative elements.

In [ ]:
nmf_model = NMF(n_epochs=50, n_factors=200, random_state=42,verbose=True)
nmf_model.fit(trainset)
nmf_predictions =nmf_model.test(testset)
nmf_rmse = accuracy.rmse(nmf_predictions)

#### SlopeOne Algorithm

Slope One is a family of algorithms used for collaborative filtering, introduced in a 2005 paper by Daniel Lemire and Anna Maclachlan. Arguably, it is the simplest form of non-trivial item-based collaborative filtering based on ratings. Their simplicity makes it especially easy to implement them efficiently while their accuracy is often on par with more complicated and computationally expensive algorithms

In [ ]:
slo_model = SlopeOne()
slo_model.fit(trainset)
slo_predictions = slo_model.test(testset)
slo_rmse=accuracy.rmse(slo_predictions)

#### CoClustering Algorithm

Biclustering, block clustering , co-clustering, or two-mode clustering is a data mining technique which allows simultaneous clustering of the rows and columns of a matrix. The term was first introduced by Boris Mirkin to name a technique introduced many years earlier, in 1972, by J. A. Hartigan.

In [ ]:
cc_model = CoClustering(n_epochs=50,random_state=42)
cc_model.fit(trainset)
cc_predictions = cc_model.test(testset)
cc_rmse=accuracy.rmse(cc_predictions)

#### Singular Value Decomposition (SVD)

The singular value decomposition (SVD) provides another way to factorize a matrix, into singular vectors and singular values. The SVD allows us to discover some of the same kind of information as the eigendecomposition.The SVD is used widely both in the calculation of other matrix operations, such as matrix inverse, but also as a data reduction method in machine learning. SVD can also be used in least squares linear regression, image compression, and denoising data.

In [ ]:
svd_model = SVD(n_epochs=50,n_factors=400,init_std_dev=0.001,random_state=42,verbose=True)
svd_model.fit(trainset)
svd_predictions = svd_model.test(testset)
svd_rmse = accuracy.rmse(svd_predictions)

#### Singular Value Decomposition plus-plus (SVDpp)

The SVD++ algorithm, an extension of SVD taking into account implicit ratings.

In [ ]:
svdpp_model = SVDpp(n_epochs=50,n_factors=400,init_std_dev=0.001,random_state=42, verbose=True)
svdpp_model.fit(trainset)
svdpp_predictions = svdpp_model.test(testset)
svdpp_rmse = accuracy.rmse(svdpp_predictions)

#### BaselineOnly algorithm

Algorithm predicting the baseline estimate for given user and item.

In [ ]:
bsl_options = {'method': 'sgd','n_epochs': 50}
blo_model = BaselineOnly(bsl_options=bsl_options,verbose=True)
blo_model.fit(trainset)
blo_predictions = blo_model.test(testset)
# Calculate RMSE
blo_rmse = accuracy.rmse(blo_predictions)

# [6. Evaluation](#eva)

We built and tested six different collaborative filtering models and compared their performance using a statistical measure known as the root mean squared error (**RMSE**), which determines the average squared difference between the estimated values and the actual value. A low RMSE value indicates high model accuracy.

### Root Mean Squared Error (RMSE):
$$RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$   

In [ ]:
rmse_scores =[nmf_rmse,slo_rmse,cc_rmse,svd_rmse,svdpp_rmse,blo_rmse]
models =['NMF','SlopeOne','CoClustering','SVD','SVD++','BaselineOnly']

accuracy_data = pd.DataFrame({'model':models,'RMSE':rmse_scores})

In [ ]:
accuracy_data.sort_values(by='RMSE')

In [ ]:
fig, ax = plt.subplots(figsize=(14,7))
sns.barplot(data=accuracy_data.sort_values(by='RMSE'), x='model', y='RMSE', palette="CMRmap", edgecolor="black", ax=ax)
ax.set_xlabel("Model")
ax.set_ylabel('RMSE Score')
ax.set_yticklabels(['{:,}'.format(int(x)) for x in ax.get_yticks().tolist()])
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2, p.get_y() + p.get_height(), round(p.get_height(),3), fontsize=12, ha="center", va='bottom')
plt.title('Model Accuracy By RMSE Score', fontsize=14)
plt.show()

###  Model Explainability and Interpretability

In the context of machine learning and artificial intelligence, <b>explainability</b> and <b>interpretability</b> are often used interchangeably. While they are very closely related, it’s worth unpicking the differences, if only to see how complicated things can get once you start digging deeper into machine learning systems.

<b>Interpretability</b> is about the extent to which a cause and effect can be observed within a system. Or, to put it another way, it is the extent to which you are able to predict what is going to happen, given a change in input or algorithmic parameters. It’s being able to look at an algorithm and go yep, I can see what’s happening here.

<b>Explainability</b>, meanwhile, is the extent to which the internal mechanics of a machine or deep learning system can be explained in human terms. 

It’s easy to miss the subtle difference with interpretability, but consider it like this: interpretability is about being able to discern the mechanics without necessarily knowing why. Explainability is being able to quite literally explain what is happening.

# [7. Conclusion](#con)

Facebook, YouTube, LinkedIn are among the most used websites on the internet today that use recommender systems. Facebook suggests us to make more friends using the 'People You May Know' section. Similarly, LinkedIn recommends you connect with people you may know, and YouTube suggests relevant videos based on your previous browsing history. All of these are recommender systems in action.

While most of the people are aware of these features, only a few know that the algorithms used behind these features are known as 'Recommender Systems'. They 'recommend' personalised content based on user's past / current preference to improve the user experience. 

We were tasked with accurately predicting unseen movie ratings gathered from thousands of users based on their historic preferences. 

Broadly, there are two types of recommendation systems: Content-Based and Collaborative filtering based as mention. In the notebook, we observation algorithms of both content-based and collaborative filtering. 

When we used the linear regression model (content-based) on the test data, it produced an RMSE score of 0.82565. However, the Singular Value Decomposition (collaborative-filtering) performed better on the test data with an <b>RMSE score of 0.80773<b> , which is our final score on the Kaggle leaderboard. 

Our [streamlit app](http://54.74.35.46:500) has interactive unsupervised learning models that can assist your business to have better customer usage by recommending items that of interest to each user. 
